In [1]:
# Download the dataset
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip
!unzip -q food-11.zip

import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# tqdm 顯示進度條的工具
from tqdm.auto import tqdm

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
To: /content/food-11.zip
100% 963M/963M [00:17<00:00, 56.5MB/s]


In [2]:
import torchvision.transforms as transforms

# 在訓練中進行數據增強
train_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),   # 隨機水平翻轉
    transforms.RandomRotation(15),       # 隨機旋轉（角度為15度）
    transforms.RandomCrop((120, 120)),   # 隨機裁剪到120x120大小
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),   # 隨機顏色調整
    transforms.ToTensor(),
])

# 測試和驗證中只進行圖像轉換
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])


In [3]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 64

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:

import torchvision.models as models
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        # 載入 ResNet-18 模型
        self.resnet = models.resnet18(pretrained=False)

        # 替換最後一層全連接層
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 11)

    def forward(self, x):
        x = self.resnet(x)
        return x


In [5]:
def get_pseudo_labels(dataset, model, threshold=0.65):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Construct a data loader.
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)

    # Iterate over the dataset by batches.
    for batch in tqdm(data_loader):
        img, _ = batch

        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)

        # ---------- TODO ----------
        # Filter the data and construct a new dataset.

    # # Turn off the eval mode.
    model.train()
    return dataset

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm

# 假設已經定義了以下函式和變數：
# get_pseudo_labels(unlabeled_set, model): 給定未標記數據和訓練好的模型，返回使用模型生成的伪標籤的數據集。
# train_set: 有標籤的訓練數據集。
# unlabeled_set: 未標記的數據集。
# valid_loader: 驗證數據的 DataLoader。
# batch_size: 批次大小。

# "cuda" 只有當有 GPU 可用時使用。
device = "cuda" if torch.cuda.is_available() else "cpu"

# 初始化模型並將其放置在指定的設備上。
model = Classifier().to(device)
model.device = device

# 對於分類任務，我們使用交叉熵作為性能衡量標準。
criterion = nn.CrossEntropyLoss()

# 初始化優化器，您可以自行微調一些超參數，如學習率。
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# 訓練的 epoch 數量。
n_epochs = 80

# 是否進行半監督學習。
do_semi = False

for epoch in range(n_epochs):
    # ---------- TODO ----------
    # 在每個 epoch 中，為半監督學習重新標記未標記數據集。
    # 然後，您可以將有標籤數據集和伪標籤數據集結合起來進行訓練。
    if do_semi:
        # 使用訓練好的模型為未標記數據獲取伪標籤。
        pseudo_set = get_pseudo_labels(unlabeled_set, model)

        # 構建新的數據集和數據加載器用於訓練。
        # 這僅在半監督學習中使用。
        concat_dataset = ConcatDataset([train_set, pseudo_set])
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

    # ---------- 訓練 ----------
    # 在訓練之前確保模型處於訓練模式。
    model.train()

    # 這些變量用於記錄訓練過程中的信息。
    train_loss = []
    train_accs = []

    # 逐批次迭代訓練數據集。
    for batch in tqdm(train_loader):

        # 一批次包含圖像數據和相應的標籤。
        imgs, labels = batch

        # 將數據傳遞到模型中（確保數據和模型在同一設備上）。
        logits = model(imgs.to(device))

        # 計算交叉熵損失。
        # 在計算交叉熵之前，我們不需要在 logits 上應用 softmax，因為它會自動完成。
        loss = criterion(logits, labels.to(device))

        # 清除先前步驟中參數中存儲的梯度。
        optimizer.zero_grad()

        # 計算參數的梯度。
        loss.backward()

        # 限制梯度的範數以實現穩定訓練。
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # 使用計算出的梯度更新參數。
        optimizer.step()

        # 計算當前批次的準確率。
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # 記錄損失和準確率。
        train_loss.append(loss.item())
        train_accs.append(acc)

    # 訓練集的平均損失和準確率是記錄值的平均值。
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # 輸出訓練信息。
    print(f"[ 訓練 | {epoch + 1:03d}/{n_epochs:03d} ] 損失 = {train_loss:.5f}, 準確率 = {train_acc:.5f}")

    # ---------- 驗證 ----------
    # 確保模型處於評估模式，以便一些模塊如 dropout 被禁用並正常工作。
    model.eval()

    # 這些變量用於記錄驗證過程中的信息。
    valid_loss = []
    valid_accs = []

    # 逐批次迭代驗證數據集。
    for batch in tqdm(valid_loader):

        # 一批次包含圖像數據和相應的標籤。
        imgs, labels = batch

        # 在驗證過程中我們不需要梯度。
        # 使用 torch.no_grad() 可以加速前向過程。
        with torch.no_grad():
            logits = model(imgs.to(device))

        # 我們仍然可以計算損失（但沒有梯度）。
        loss = criterion(logits, labels.to(device))

        # 計算當前批次的準確率。
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # 記錄損失和準確率。
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # 整個驗證集的平均損失和準確率是記錄值的平均值。
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # 輸出驗證信息。
    print(f"[ 驗證 | {epoch + 1:03d}/{n_epochs:03d} ] 損失 = {valid_loss:.5f}, 準確率 = {valid_acc:.5f}")


100%|██████████| 49/49 [00:35<00:00,  1.40it/s]


[ 訓練 | 001/080 ] 損失 = 2.28793, 準確率 = 0.18112


100%|██████████| 11/11 [00:05<00:00,  1.96it/s]


[ 驗證 | 001/080 ] 損失 = 2.87774, 準確率 = 0.17670


100%|██████████| 49/49 [00:30<00:00,  1.63it/s]


...

[ 驗證 | 079/080 ] 損失 = 2.42373, 準確率 = 0.48722


100%|██████████| 49/49 [00:28<00:00,  1.72it/s]


[ 訓練 | 080/080 ] 損失 = 0.15401, 準確率 = 0.94834


100%|██████████| 11/11 [00:05<00:00,  1.97it/s]

[ 驗證 | 080/080 ] 損失 = 2.48382, 準確率 = 0.49006


In [7]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

100%|██████████| 53/53 [00:29<00:00,  1.80it/s]


In [8]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")